# Парсинг pdf-документов (выделение текста)

In [1]:
import io
import re
import os, wget
import requests
import glob
from tqdm.notebook import tqdm

from tika import parser

UNIVS = ['АлтГУ', 'АлтГТУ', 'ВолГУ', 'ИМСИТ']
CURRPATH = os.getcwd()
DATAPATH = '\\'.join(CURRPATH.split('\\')[:-1]) + '\\new-data'
FILESPATH = DATAPATH + '\\new-files'

try:
    os.makedirs(FILESPATH)
except:
    pass

In [2]:
regex_general = [
    r'Форма обучения', r' опк ', r'в  ?т[.] ?ч[.]',
    r'Объем дисциплины', r'и  ?т[.] ?п[.]', r'и  ?т[.] ?д[.]',
    r'Форма промежуточной аттестации',
    r'Контрольная работа', r'трудоемкость \d{1,3}',
    r'\d{1,3} часов', r'\d{1,3} часах', r'\d{1,3} час[аы]?',
#     r' Зачет ', r'Экзамен[аы]?', r'Очная', r'Заочная',
    r'[cс]тр[.] ?\d\d{0,2}-?\d{0,3}',
    r'\d\d[.]\d\d[.]\d\d', r'\d\d[.]\d\d[.]\d\d\d\dг?',
    r'\d\d[.]\s{1,8}[.]\d\d\d\dг?', r'Б\d?([.]дв)?[.]\d\d([.]\d\d)?',
    r'Б\d', r'-ы', r'-[аеиы]х', r'\([аеиы]х\)', r'-[оияуеы]й', r'-[оая]я', r'-[аыяое]ми',
    r'-[оеы]ва?', r'\d\d?\-го', r'\d\d?\-ы[ей]' r'-[ое]го', r'-[ое]му',
    r'-[ыиое]м', r'-[ую]ю', r'-[оыие]е',
    r'Тема ?\d\d?[.]?', r'Часть ?\d\d?[.]?', r'Модуль \d\d?[.]?', r'Модуль [IVXCDL]*',
#     r'\s\d\d?', r'\d[.]',
    r'https?://.*?[.]com/?',
    r'https?://.*?[.]ru/?',
    r'https?://.*?[.]org/?',
    r'www.*?[.]com/?',
    r'www.*?[.]ru/?',
    r'www.*?[.]org/?',
    r'\[.*?\]',
#     r'\(\d{1,3}\)',
#     r'\(\d{1,3}, \d{1,3}\)',
    r'\s\-', r'\-\s', r'\d[.]\d',
    r'[.,:;\"\'«»\(\)*!?\/\\–—№¶•¬·“”„…>%=’]',
    r'Семестр ', r'Раздел ', r' \d{1,3} ', r' РФ ',
]

regex_imsit = [
    r'.*Уникальный программный ключ:? \S{60,70} ',
    r'Практические занятия:.*',
    r'Лекционные занятия:.*',
    r'Лабораторные работы.*',
    r'\d{1,3} ?ч/\d{1,3} ?з[.]е[.]',
    r'МЕСТО ДИСЦИПЛИНЫ В СТРУКТУРЕ ООП  Цикл (раздел) ООП: .*  2[.]1',
    r'.*ЦЕЛИ ОСВОЕНИЯ ДИСЦИПЛИНЫ',
    r'КОМПЕТЕНЦИИ ОБУЧАЮЩЕГОСЯ ФОРМИРУЕМЫЕ В РЕЗУЛЬТАТЕ ОСВОЕНИЯ ДИСЦИПЛИНЫ МОДУЛЯ',
    r'Общая трудоемкость изучения.*',
    r'(УП)? ПедОбр2 3[+]{2}Иняз22[.]?plx',
    r'суюности',
    r'ФГОС ВО',
    r'\d\d ?х ?\d\d ?м',
    r'Б.М.\d\d.\d\d',
    r'В.ДЭ.\d\d?',
    r'[cс]{3,}', r'[уy]{3,}',
    r'Формы проведения.*',
    r'Используемые инструментальные.*',
    r'Место\s*?дисциплины\s*?в\s*?структуре\s*?ОПОП.*\(основные блоки, темы\)',
]

regex_volgu = [    
    r'.*Цель и задачи изучения дисциплины',
    r'Задачи дисциплины',
    r'Место дисциплины в структуре ОПОП.*\d. Содержание дисциплины',
    r'Выпускник *?должен *?обладать *?следующими *?общепрофессиональными *?компетенциями',
    r'Выпускник *?должен *?обладать *?следующими *?универсальными *?компетенциями',
    r'Процесс  изучения  дисциплины  направлен.*ФГОС (ВО)?(НОО)?',
    r'Требования к результатам освоения дисциплины',
    r'Студент должен',
    r'Разработчик.*',
]

regex_altgtu = [
    r'Разработал:.*',
    r'.*В результате освоения дисциплины обучающийся должен обладать следующими компетенциями:',
    r'Контрольный опрос и тест по модулю',
    r'Синтаксис текста',
]

regex_altgu = [
    r'Б\d?[.](В[.])?(ДВ[.])?\d\d?.\d\d?',
    r'.*Цель изучения дисциплины',
#     r'Форма промежуточной аттестации.*',
#     r'Место дисциплины в учебном плане .*?  Формируемые',
    r'Используемые информационные, инструментальные и программные средства.*',
    r'Виды учебной работы.*',
    r'Целями изучения дисциплины являются',
]

REGEXES = {
    'ИМСИТ' : regex_imsit,
    'ВолГУ' : regex_volgu,
    'АлтГТУ' : regex_altgtu,
    'АлтГУ' : regex_altgu,
}

In [3]:
def parse(file):
    raw = parser.from_file(file)
    text = raw['content'].encode("ansi", "ignore").decode('ansi')
    text = re.sub(r'-\n\n?', '', text)
    text = text.replace('ё', 'е').replace('\n-', '').replace('\n', ' ').replace('-\t', ' ').replace('\t', ' ')
    return text

def brush(text, regex):
    for reg in regex:
        text = re.sub(reg, ' ', text, flags=re.IGNORECASE)
    for reg in regex_general:
        text = re.sub(reg, ' ', text, flags=re.IGNORECASE)
    return text


def clean_text(text, regex):
    text = re.sub(r'[ОПКИДУ]{2,3}(--)?[-– ]?( [-–])?([-–] )?( [-–] )?\d[.]?\d?', ' ', text)
    text = re.sub(r' [ОПКИДУ]{2,3} ', ' ', text)
    text = brush(text, regex)
    text = re.sub(r'[().,:;]', '', text)
    text = re.sub(r'\ {2,}', ' ', text).lstrip().rstrip()
    return text


def save(file, text, split_by='\\'):
    splitted = file.split(split_by)
    splitted[-1] = splitted[-1].replace('.pdf', '.txt')
    new_path = FILESPATH + '\\' + '\\'.join(splitted[-3:-1])
    
    try:
        os.makedirs(new_path)
    except:
        pass
    
    new_path += '\\' + splitted[-1]
    
    file_ = open(new_path, "w+")
    file_.write(text)
    file_.close()

In [4]:
for U in tqdm(UNIVS):
    regex = REGEXES[U]
    for file in glob.glob(DATAPATH + '\{}\*\*.pdf'.format(U)):
        try:
            text = parse(file)
        except Exception as e:
            print('parsing error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue
        try:
            text = clean_text(text, regex)
        except Exception as e:
            print('cleaning error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue
        try:
            save(file, text)
        except Exception as e:
            print('saving error for {}:'.format('\\'.join(file.split('\\')[-3:])), e)
            print()
            continue

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# Выкладки для обзора

# files_ = []
# for file in glob.glob(DATAPATH + '\\???\\*\\*.pdf'):
#     files_ += [file]

In [6]:
# from IPython.display import display, clear_output

# files = files_[:]
# for i, file in enumerate(files):
#     clear_output(wait=False)
#     print(file.replace('\\', '\\\\'))    
#     raw = parser.from_file(file)
#     text = raw['content'].encode("ansi", "ignore").decode('ansi')
#     text = re.sub(r'-\n\n?', '', text)
#     text = text.replace('ё', 'е').replace('\n-', '').replace('\n', ' ').replace('\t', ' ')
#     display(text)
#     print()
#     display(clean_text(text, REGEXES[file.split('\\')[-3]])))
#     print(i)
#     _ = input()